## Data Preprocessing

In [ ]:
!unzip /content/drive/MyDrive/NLP_HW2/History.zip -d /content/drive/MyDrive/NLP_HW2/History

Archive:  /content/drive/MyDrive/NLP_HW2/History.zip
  inflating: /content/drive/MyDrive/NLP_HW2/History/EVENTS.json  
  inflating: /content/drive/MyDrive/NLP_HW2/History/filtered_extracted_contents.json  
  inflating: /content/drive/MyDrive/NLP_HW2/History/Treaties.json  
  inflating: /content/drive/MyDrive/NLP_HW2/History/WAR_1.json  
  inflating: /content/drive/MyDrive/NLP_HW2/History/WAR_2.json  


In [ ]:
import json
from tqdm import tqdm


In [ ]:
import openai

api_key1 = ""
api_key2 = ""
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

client = openai.OpenAI(
    api_key=api_key1,
    base_url=base_url
)

LLM_API_MODEL = "gemini-2.0-flash-lite"

def ask_gemini(user_input):
  global client
  try:
    response = client.chat.completions.create(
                model=LLM_API_MODEL,
                messages=[{"role": "user", "content": user_input}]
            )
  except:
    print("*** client2 activated")
    client = openai.OpenAI(
        api_key=api_key2,
        base_url=base_url
    )
    response = client.chat.completions.create(
                model=LLM_API_MODEL,
                messages=[{"role": "user", "content": user_input}]
            )
  content = response.choices[0].message.content
  return content

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "unsloth/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

def ask_llama(question):
  messages = [
   {"role": "user", "content": question}
  ]
  outputs = pipe(
      messages,
      max_new_tokens=2048
  )
  return (outputs[0]["generated_text"][-1]['content'])

KeyboardInterrupt: 

### Using LLM to convert JSON into readable format

In [ ]:
PROMPT = """
You are a skilled language model specialized in converting structured JSON records into fluent, natural Persian text.

Your task is to read each JSON object and generate a paragraph that:
- Accurately includes key information from the data.
- Is readable and natural for Persian speakers.
- Sounds like it was written by a human (not robotic or repetitive).
- Emphasizes diverse sentence structures across generations.
- JUST use PERSIAN! Do not use any other language

{{shot}}

### Input JSON:
{{json_str}}
"""

In [ ]:
name = "EVENTS"

shot = """
### Example Input JSON:
{
  "title": "ولادت سید ابراهیم ضیاء الواعظین مدیر مسئول روزنامه ایران آزاد در شیراز در ۲۲ خرداد سال ۱۲۳۷ ش (۲۹ شوال ۱۲۷۴ ق)",
  "description": "روزنامه ایران آزاد روزنامه‌ای بود با گرایش سیاسی، اجتماعی که از ۱۳۰۰ ش در تهران منتشر می‌شد. ایران آزاد از همان ابتدا با انتشار مقالات تند مورد توجه مردم قرار گرفت و به طرز کم‌سابقه‌ای خواننده پیدا کرد و به دلیل چاپ مقالاتی علیه حکومت چند بار توقیف گردید و مدیر مسئول آن تبعید شد. روزنامه ایران آزاد با مدیریت و صاحب‌امتیازی سید ابراهیم ضیاء الواعظین انتشار می‌یافت. سید ابراهیم ضیاء الواعظین در ۲۲ خرداد ۱۲۳۷ در شیراز دیده به جهان گشود.",
  "period": {
    "start_year": "۲۸۲۴۹۶",
    "end_year": "۲۸۲۴۹۶"
  },
  "location": {
    "province": "شیراز",
    "city": "شیراز",
    "coordinates": {
      "latitude": 29.61,
      "longitude": 52.5425
    }
  },
  "refrences": {
    "title": "روز شمار تاریخ معاصر ایران، ج ۲، ص ۲۹۶",
    "author": "حسن فراهانی",
    "year": "۱۳۸۵"
  },
  "source": {
    "title": "ولادت ضیاء الواعظین",
    "author": "پایگاه جامع تاریخ",
    "url": "https://tarikh.inoor.ir/fa/event/page/1K355M/ولادت_ضیاء_الواعظین"
  }
}

### Output Text:
سید ابراهیم ضیاء الواعظین، روزنامه‌نگار و فعال سیاسی، در ۲۲ خرداد سال ۱۲۳۷ هجری شمسی در شیراز دیده به جهان گشود. او مدیرمسئول و صاحب‌امتیاز روزنامه "ایران آزاد" بود که از سال ۱۳۰۰ شمسی در تهران با رویکردی سیاسی و اجتماعی منتشر می‌شد. این روزنامه با انتشار مطالب انتقادی علیه حکومت، توجه گسترده‌ای را به خود جلب کرد و چندین بار توقیف و سانسور شد. ضیاء الواعظین نیز به دلیل مطالب جنجالی روزنامه‌اش، تبعید گردید. اطلاعات بیشتر در این زمینه در کتاب "روزشمار تاریخ معاصر ایران" اثر حسن فراهانی آمده است.
"""

PROMPT = PROMPT.replace("{{shot}}", shot)

data = json.load(open(f'/content/drive/MyDrive/NLP_HW2/History/{name}.json'))

output_file = f"/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/{name}.txt"

with open(output_file, "a", encoding="utf-8") as f:
  for event_record in data[50:]:
    json_str = str(event_record["data"])
    res = ask_gemini(PROMPT.replace("{{json_str}}", json_str))
    f.write(res.strip())
    f.write("\n\n")

In [ ]:
name = "Treaties"

shot = """
### Example Input JSON:
{
  "title": "پیمان مونترال",
  "description": "پیمان مونترال در سال ۱۹۸۷ انجام شد و پس از نخستین جلسه در هلسینکی، مه ۱۹۸۹، در ۱۶ سپتامبر ۱۹۸۹ به اجرا درآمد.",
  "period": {
      "year": "۱۹۸۷"
  },
  "location": {
      "city": "مونترال",
      "coordinates": {
          "latitude": 0,
          "longitude": 0
      }
  },
  "causes": "دلیل امضای این قرارداد: محافظت از لایه ازون و کاهش تولید مواد که باعث تخریب آن می‌شوند.",
  "belligerents": [
      "پیمان مونترال",
      "کنوانسیون وین برای محافظت از لایه ازون"
  ],
  "result": "پیمان مونترال در سال ۱۹۸۷ امضا شد و پس از نخستین جلسه در هلسینکی، مه ۱۹۸۹، در ۱۶ سپتامبر ۱۹۸۹ به اجرا درآمد.",
  "impact": "تأثیرات: کاهش سوراخ ازون در قطب جنوب، بهبود وضعیت لایه ازون تا سال ۱۹۸۰، موفق‌ترین توافق بین‌المللی تا به امروز، نمونه‌ای از همکاری‌های بین‌المللی، نخستین معاهده‌های جهانی تصویب شده در تاریخ سازمان ملل متحد، فرایند سیاست‌گزاری در مقیاس جهانی.",
  "historical_significance": "پیمان مونترال به دلیل اهمیت تاریخی و موفقیت آن در کاهش سوراخ ازون، به عنوان نمونه‌ای از همکاری‌های بین‌المللی استثنایی شناخته می‌شود. این معاهده به نخستین معاهده‌های جهانی تصویب شده در تاریخ سازمان ملل متحد تبدیل شده است.",
  "references": [
      "<TBD>",
      {
          "title": "\"Kigali Amendment Enters into Force, Bringing Promise of Reduced Global Warming | News | SDG Knowledge Hub | IISD\"",
          "author": "Hub, IISD's SDG Knowledge.\"Kigali Amendment Enters into Force, Bringing Promise of Reduced Global Warming | News | SDG Knowledge Hub | IISD\"(به انگلیسی). Retrieved2019-03-07.",
          "year": "Unknown"
      },
      {
          "title": "\"Deal reached on HFC greenhouse gases\"",
          "author": "McGrath, Matt (15 October 2016).\"Deal reached on HFC greenhouse gases\"– via www.bbc.co.uk.",
          "year": "Unknown"
      }
  ],
  "source": {
      "title": "پیمان مونترال",
      "author": "Wikipedia",
      "publication_date": "Unknown",
      "url": "https:\/\/fa.wikipedia.org\/wiki\/%D9%BE%DB%8C%D9%85%D8%A7%D9%86_%D9%85%D9%88%D9%86%D8%AA%D8%B1%D8%A7%D9%84"
  }
}

### Output Text:
«پیمان مونترال»، یکی از مهم‌ترین توافق‌نامه‌های زیست‌محیطی جهان، در سال ۱۹۸۷ در شهر مونترال کانادا منعقد شد. این پیمان با هدف اصلی محافظت از لایه ازون و کاهش تولید مواد مخرب آن مانند CFCها، به‌عنوان یکی از موفق‌ترین معاهده‌های بین‌المللی در تاریخ سازمان ملل شناخته می‌شود. پس از نخستین جلسه پیگیری در هلسینکی در مه ۱۹۸۹، این توافق در ۱۶ سپتامبر همان سال به مرحله اجرا درآمد. این معاهده در کنار کنوانسیون وین برای محافظت از لایه ازون، الگویی از همکاری جهانی برای مقابله با بحران‌های زیست‌محیطی ارائه داد. پیامدهای مثبت پیمان شامل کاهش قابل توجه سوراخ ازون در قطب جنوب و روند بهبود وضعیت لایه ازون تا سطح سال ۱۹۸۰ است. پیمان مونترال در تاریخ به‌عنوان نمونه‌ای استثنایی از سیاست‌گذاری جهانی و اجماع بین‌المللی برای حفظ محیط زیست ثبت شده است."""

PROMPT = PROMPT.replace("{{shot}}", shot)

data = json.load(open(f'/content/drive/MyDrive/NLP_HW2/History/{name}.json'))
# len(data)
output_file = f"/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/{name}.txt"

with open(output_file, "a", encoding="utf-8") as f:
  for event_record in tqdm(data[86:]):
    json_str = str(event_record["data"])
    res = ask_gemini(PROMPT.replace("{{json_str}}", json_str))
    f.write(res.strip())
    f.write("\n\n")

100%|██████████| 41/41 [01:26<00:00,  2.11s/it]


In [ ]:
name = "WAR_1"

shot = """
### Example Input JSON:
{'title': 'تهاجم آشور به مادها',
 'description': 'نامشخص',
 'period': {'start_year': '', 'end_year': ''},
 'position': 'نامشخص',
 'causes': [],
 'belligerents': {'names': '', 'leaders': ''},
 'result': 'شکست: آشوریان، پادشاهی\u200cها و دولت-شهرهای غرب ایران را تصرف کردند.',
 'casualties': {},
 'impact': 'نامشخص',
 'historical_significance': 'نامشخص',
 'references': [],
 'source': {'title': 'تهاجم آشور به مادها',
  'author': 'Wikipedia',
  'publication_date': 'Unknown',
  'url': 'https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%A7%D8%AC%D9%85_%D8%A2%D8%B4%D9%88%D8%B1_%D8%A8%D9%87_%D9%85%D8%A7%D8%AF%D9%87%D8%A7'}}

### Output Text:
تهاجم آشور به مادها» یکی از رویدادهای تاریخی مهم در دوران باستان ایران به‌شمار می‌رود. با وجود ناشناخته بودن جزئیات دقیق این تهاجم از جمله زمان دقیق وقوع یا دلایل آن، می‌دانیم که این حمله منجر به شکست مادها شد. در نتیجه، آشوریان توانستند بخش‌هایی از غرب ایران، از جمله پادشاهی‌ها و دولت‌شهرها را به تصرف خود درآورند. اطلاعات بیشتری درباره رهبران، انگیزه‌ها یا پیامدهای بلندمدت این حمله در دست نیست، اما این رویداد به عنوان یکی از برخوردهای تأثیرگذار بین تمدن‌های منطقه شناخته می‌شود.
"""

PROMPT = PROMPT.replace("{{shot}}", shot)

data = json.load(open(f'/content/drive/MyDrive/NLP_HW2/History/{name}.json'))

output_file = f"/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/{name}.txt"

with open(output_file, "a", encoding="utf-8") as f:
  for event_record in tqdm(data):
    json_str = str(event_record["data"])
    res = ask_gemini(PROMPT.replace("{{json_str}}", json_str))
    f.write(res.strip())
    f.write("\n\n")

100%|██████████| 100/100 [05:30<00:00,  3.30s/it]


In [ ]:
name = "WAR_2"

shot = """
### Example Input JSON:
{'title': 'تهاجم آشور به مادها',
 'description': 'نامشخص',
 'period': {'start_year': '', 'end_year': ''},
 'position': 'نامشخص',
 'causes': [],
 'belligerents': {'names': '', 'leaders': ''},
 'result': 'شکست: آشوریان، پادشاهی\u200cها و دولت-شهرهای غرب ایران را تصرف کردند.',
 'casualties': {},
 'impact': 'نامشخص',
 'historical_significance': 'نامشخص',
 'references': [],
 'source': {'title': 'تهاجم آشور به مادها',
  'author': 'Wikipedia',
  'publication_date': 'Unknown',
  'url': 'https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%A7%D8%AC%D9%85_%D8%A2%D8%B4%D9%88%D8%B1_%D8%A8%D9%87_%D9%85%D8%A7%D8%AF%D9%87%D8%A7'}}

### Output Text:
تهاجم آشور به مادها» یکی از رویدادهای تاریخی مهم در دوران باستان ایران به‌شمار می‌رود. با وجود ناشناخته بودن جزئیات دقیق این تهاجم از جمله زمان دقیق وقوع یا دلایل آن، می‌دانیم که این حمله منجر به شکست مادها شد. در نتیجه، آشوریان توانستند بخش‌هایی از غرب ایران، از جمله پادشاهی‌ها و دولت‌شهرها را به تصرف خود درآورند. اطلاعات بیشتری درباره رهبران، انگیزه‌ها یا پیامدهای بلندمدت این حمله در دست نیست، اما این رویداد به عنوان یکی از برخوردهای تأثیرگذار بین تمدن‌های منطقه شناخته می‌شود.
"""

PROMPT = PROMPT.replace("{{shot}}", shot)

data = json.load(open(f'/content/drive/MyDrive/NLP_HW2/History/{name}.json'))

output_file = f"/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/{name}.txt"

with open(output_file, "a", encoding="utf-8") as f:
  for event_record in tqdm(data):
    json_str = str(event_record["data"])
    res = ask_gemini(PROMPT.replace("{{json_str}}", json_str))
    f.write(res.strip())
    f.write("\n\n")

100%|██████████| 7/7 [00:19<00:00,  2.86s/it]


In [ ]:
name = "filtered_extracted_contents"

PROMPT = PROMPT.replace("{{shot}}", "")

data = json.load(open(f'/content/drive/MyDrive/NLP_HW2/History/{name}.json'))

output_file = f"/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/{name}2.txt"

with open(output_file, "a", encoding="utf-8") as f:
  for event_record in tqdm(data[121:200]):
    json_str = str(event_record)
    res = ask_llama(PROMPT.replace("{{json_str}}", json_str))
    # print("\n*Event record: ", json_str)
    # print("\n*Res: ", res.strip())
    res = res.replace("\n", "")
    f.write(res.strip())
    f.write("\n\n")

 21%|██        | 21/100 [09:36<30:13, 22.96s/it]

### Diverse Question Generation

In [ ]:
import os

folder_path = '/content/drive/MyDrive/NLP_HW2/History/ReadableFormats'
all_sentences = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt') and "filtered_extracted_contents" not in filename:
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                sentence = line.strip()
                if sentence:  # Ignore empty lines
                    all_sentences.append(sentence)

In [ ]:
len(all_sentences)

342

In [ ]:
keywords = ["اسرائیل", "کوروش", "رضاشاه","مشروطه", "پهلوی", "ترکمانچای",
            "مظفرالدین شاه", "هسته‌ای", "ایالات متحده", "داعش", "ساسانیان", "عثمانی"]
clusters = {keyword:[] for keyword in keywords}
for key in clusters.keys():
  for i in all_sentences:
    if  key in i:
      clusters[key].append(i)


In [ ]:
for i , j in clusters.items():
  print(i, len(j))


اسرائیل 8
کوروش 9
رضاشاه 4
مشروطه 16
پهلوی 22
ترکمانچای 4
مظفرالدین شاه 6
هسته‌ای 4
ایالات متحده 21
داعش 3
ساسانیان 20
عثمانی 33


In [ ]:
questions = []

In [ ]:
clusters["اسرائیل"][4]

'کوروش بزرگ، بنیان\u200cگذار و نخستین شاهنشاه امپراتوری هخامنشی، شخصیتی برجسته در تاریخ باستان است که در حدود سال\u200cهای ۶۰۰ تا ۵۳۰ پیش از میلاد می\u200cزیست. او که با نام\u200cهای کوروش دوم و کوروش کبیر نیز شناخته می\u200cشود، در ابتدا با ایشتوویگو، پادشاه ماد، به نبرد پرداخت و سپس با فتح هگمتانه، پایتخت ماد، قدرت خود را تثبیت کرد. کوروش پس از شکست کرزوس، شاه لیدیه، و فتح سارد، به توسعه قلمرو خود ادامه داد و سرزمین\u200cهای وسیعی از آسیا را تحت سلطه خود درآورد. در سال ۵۳۹ پیش از میلاد، کوروش به بابل حمله کرد و با فتح این شهر، امپراتوری خود را گسترش داد. این پیروزی زمینه\u200cساز بازگشت یهودیان تبعیدی به سرزمین اسرائیل شد و دستور بازسازی پرستش\u200cگاه اورشلیم و بازگرداندن اشیای گرانبهای آن را صادر کرد. استوانه کوروش، سندی تاریخی از این دوران است که پس از فتح بابل نگاشته شده است. او یک نظامی نابغه بود که در سال\u200cهای ۵۵۰ تا ۵۳۹ پیش از میلاد، پیروزی\u200cهای متعددی کسب کرد و قلمرو حکومتش از شرق تا رود سند و از غرب تا آسیای کوچک گسترش یافت.'

In [ ]:
for i in clusters["عثمانی"]:
  print(i[:100])

رضا قلی خان نظام السلطنه در سال ۱۲۸۴ قمری چشم به جهان گشود. او فرزند حیدر قلی خان برهان الدوله بود و
در سال ۱۸۹۰، قانون کنفرانس بروکسل با هدف پایان دادن به تجارت برده و بهبود شرایط زندگی مردم بومی تدوی
به فرمان شاه عباس یکم صفوی، سفارت ایران در سال ۱۵۹۹ میلادی (۹۷۸ خورشیدی) به منظور یافتن متحدی علیه ا
در سال ۹۸۸ هجری قمری، ایران به منظور یافتن متحدی علیه امپراتوری عثمانی، اقدام به اعزام سفیرانی به ار
در سال ۱۷۴۶ میلادی، عهدنامه گردان در نزدیکی شهر قزوین به امضا رسید. این پیمان که به منظور پایان دادن
پیمان آماسیه، که در سال ۱۵۵۵ میلادی در شهر آماسیه به امضا رسید، نقطه عطفی در روابط بین ایران صفوی و 
در سال ۱۵۹۰ میلادی، پیمان فرهاد پاشا، که با نام عهدنامه استانبول نیز شناخته می‌شود، به امضا رسید. ای
در ۹ اکتبر سال ۱۸۷۴، معاهده برن در سوئیس به امضا رسید. این پیمان که با هدف ایجاد اتحادیه پستی عمومی 
در سال ۱۷۳۶، معاهده‌ای مهم با عنوان معاهده استانبول میان ایران و امپراتوری عثمانی به امضا رسید. این 
عهدنامه زهاب، که در سال ۱۶۳۹ میلادی در قصر شیرین به امضا رسید، نقطه‌ی عطفی در روابط ایران و

In [ ]:
questions2 = []

In [ ]:
import re
import json

def extract_qa(text):
    """
    Extracts question and answer from a Persian Q&A formatted string.
    Returns a dictionary with 'question' and 'answer'.
    """
    match = re.search(r'سؤال[:：]?\s*(.*?)\s*پاسخ[:：]?\s*(.*)', text, re.DOTALL)
    if not match:
        return {"text": text}  # Return None if format doesn't match
    question = match.group(1).strip()
    answer = match.group(2).strip()
    return {
        "question": question,
        "answer": answer
    }

In [ ]:
from itertools import combinations

start_iterating = False
r = 0
for k , j in clusters.items():
  r += 1
  print(k)
  if k == "ایالات متحده":
        start_iterating = True
  if start_iterating:
    comb_list = [(j[i], j[i+1]) for i in range(0, len(j) - 1, 2)]
    for a,b in comb_list:
      context = ""
      context += "\n 1 بخش"
      context += f"{a}\n"
      context += "\n 2 بخش"
      context += f"{b}\n"

      prompt = f""" یک سوال درباره {k} از محتوایی که در اختیارت قرار می دهم مطرح کن. به گونه ای که برای پاسخ به آن نیاز به بخش های مختلف محتوا باشد.
      محتوا:\n{context}
      فقط سوال نهایی و جواب آن را به فرمت زیر برگردان و در سوال از عبارت هایی مثل "بر اساس اطلاعات ارائه شده ..." استفاده نکن
      سؤال:
      پاسخ:
      """
      res = ask_gemini(prompt)
      print("**********************", k)
      formated_res = extract_qa(res)
      print(formated_res)
      questions.append(res)
      questions2.append(formated_res)


اسرائیل
کوروش
رضاشاه
مشروطه
پهلوی
ترکمانچای
مظفرالدین شاه
هسته‌ای
ایالات متحده
********************** ایالات متحده
{'question': 'ایالات متحده آمریکا در کدام یک از پیمان های ذکر شده، عضو بود و چه اهدافی را دنبال می کرد؟', 'answer': 'ایالات متحده آمریکا عضو پیمان بریان–کلوگ بود که هدف آن جلوگیری از وقوع جنگ بود. آمریکا در سنتو عضو نبود، ولی از این پیمان حمایت می کرد.'}
********************** ایالات متحده
{'question': 'ایالات متحده آمریکا در چه رویدادهای بین\u200cالمللی در اواخر قرن نوزدهم میلادی نقش داشت و این رویدادها چه تأثیری بر عرصه بین\u200cالمللی داشتند؟', 'answer': 'ایالات متحده در سال ۱۸۹۰ قانون کنفرانس بروکسل را امضا کرد که هدف آن پایان دادن به تجارت برده و بهبود شرایط زندگی بومیان بود. همچنین، در سال ۱۸۹۹ در کنوانسیون\u200cهای لاهه شرکت کرد که هدف آن\u200cها رفع اشکالات نظام داوری بین\u200cالمللی و تشویق دولت\u200cها به امضای موافقت\u200cنامه\u200cهای بین\u200cالمللی بود. این اقدامات نشان\u200cدهنده مشارکت ایالات متحده در تلاش\u200cهای بین\u200cالمللی برای ایجاد صلح و بهبود شرا

RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.0-flash-lite', 'location': 'global'}, 'quotaValue': '30'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '0s'}]}}]

In [ ]:
len(questions2)

64

In [ ]:
for q in questions2[20:22]:
  print(q)

{'question': 'نقش دو چهره سیاسی رضا قلی هدایت و نصرالله خان خلعتبری در دوران پهلوی چه بود و کدام یک زودتر درگذشت؟', 'answer': 'رضا قلی هدایت در دوران پهلوی وزیر بود و در تشکیل حزب اراده ملی نقش داشت. نصرالله خان خلعتبری نیز در دوران پهلوی فعال بود. رضا قلی هدایت در سال ۱۳۲۴ درگذشت. با توجه به تاریخ تولد نصرالله خان خلعتبری و عدم ذکر تاریخ وفات او در متن، مشخص نیست کدام یک زودتر درگذشت.'}
{'question': 'با توجه به اطلاعات موجود، دو شخصیت سیاسی که در دوران پهلوی فعالیت داشتند را نام ببرید و محل تولد هر یک را مشخص کنید.', 'answer': 'دو شخصیت سیاسی که در دوران پهلوی فعالیت داشتند، داود مفتاح السلطنه که در تهران متولد شد و میرزا علی\u200cمحمد خان بنی\u200cآدم ملقب به شریف\u200cالدوله که در کاشان چشم به جهان گشود.'}


In [ ]:
with open("/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/QA_records_2.json", "w", encoding="utf-8") as f:
  json.dump(questions2, f, ensure_ascii=False, indent=2)

In [ ]:
import re

def qan_parser(sentence: str, qa_text: str) -> dict:
    """
    Converts a sentence and corresponding Q&A pairs (formatted in text)
    into a structured JSON-like dictionary.

    Parameters:
        sentence (str): The original text the questions are based on.
        qa_text (str): Multiline string containing questions and answers.
                       Format:
                       سؤال: ...
                       پاسخ: ...

    Returns:
        dict: A dictionary with the sentence and a list of question-answer pairs.
    """
    qas = []
    qa_pairs = re.findall(r"سؤال:\s*(.*?)\s*پاسخ:\s*(.*?)(?=(?:\n\s*سؤال:|\Z))", qa_text, flags=re.DOTALL)

    for question, answer in qa_pairs:
        qas.append({
            "question": question.strip(),
            "answer": answer.strip()
        })

    return {
        "sentence": sentence.strip(),
        "Q&A": qas
    }


In [ ]:
QUESTION_GEN_PROMPT = """
You are a historical data question generator. Given the following one-line summary of a historical event, generate **5 diverse Persian (Farsi) question-answer pairs** that follow these rules:

- Use various **wh-questions** such as "چه کسی؟ چه چیزی؟ چه زمانی؟ چرا؟ چگونه؟"
- Include different **question types**: factual, reasoning, cause-effect, and inference-based.
- Ensure the questions span multiple fields: title, date, cause, result, impact, key figures, and significance.
- Write questions and answers clearly and concisely in natural, formal Persian (Farsi).
- Do not use emojis or bullet points.
- DO not generate question form references of information.
- Output format must be:
  سؤال: [question]
  پاسخ: [answer]

Here is the historical event summary:
{text}
"""
filename = "/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/QA_records.json"

for sentence in tqdm(all_sentences[:10]):
  response = ask_gemini(QUESTION_GEN_PROMPT.replace('{text}', sentence))

  rec = qan_parser(sentence, response)

  print(rec)
  with open(filename, "a", encoding="utf-8") as f:
    json.dump(rec, f, ensure_ascii=False, indent=2)


  1%|          | 1/181 [00:02<08:40,  2.89s/it]

{'sentence': 'در تاریخ ۲۲ خرداد سال ۱۲۳۷ هجری شمسی در شهر شیراز، سید ابراهیم ضیاء الواعظین چشم به جهان گشود. وی بعدها به عنوان مدیر مسئول روزنامه "ایران آزاد" شناخته شد، روزنامه\u200cای که از سال ۱۳۰۰ شمسی در تهران با موضوعات سیاسی و اجتماعی منتشر می\u200cشد. این روزنامه که به دلیل چاپ مقالات تند و انتقادی از همان ابتدا مورد توجه فراوان قرار گرفت، با تیراژی بالا خوانندگان زیادی را جذب کرد. اما به دلیل همین انتقادات، چند بار توقیف شد و مدیر مسئول آن نیز تبعید گردید. برای کسب اطلاعات بیشتر در این باره می\u200cتوانید به کتاب "روز شمار تاریخ معاصر ایران" نوشته حسن فراهانی مراجعه فرمایید.', 'Q&A': [{'question': 'سید ابراهیم ضیاء الواعظین چه زمانی و در کجا متولد شد؟', 'answer': 'سید ابراهیم ضیاء الواعظین در تاریخ ۲۲ خرداد سال ۱۲۳۷ هجری شمسی در شهر شیراز به دنیا آمد.'}, {'question': 'روزنامه "ایران آزاد" به چه موضوعاتی می\u200cپرداخت و چه تأثیری بر مخاطبان داشت؟', 'answer': 'روزنامه "ایران آزاد" با موضوعات سیاسی و اجتماعی منتشر می\u200cشد و با چاپ مقالات تند و انتقادی، توجه بسیاری از مخاطبان 

  1%|          | 2/181 [00:05<08:01,  2.69s/it]

{'sentence': 'آیت\u200cالله شیخ عبدالکریم حائری یزدی، بنیانگذار حوزه علمیه قم و از علمای برجسته شیعه، در سال ۱۲۷۶ قمری در روستای مهرجرد از توابع میبد یزد چشم به جهان گشود. ایشان تحصیلات مقدماتی را در اردکان آغاز کرد و سپس در یزد به فراگیری علوم دینی پرداخت. برای ادامه تحصیل راهی عراق شد و در حوزه\u200cهای علمیه سامرا و نجف از محضر اساتید بزرگی بهره برد. پس از درگذشت میرزای شیرازی بزرگ، به نجف رفت و به درس آخوند خراسانی پیوست. آیت\u200cالله حائری در کربلا نیز به تربیت طلاب و تشکیل حوزه درسی اهتمام ورزید و در نهایت، پس از رحلت مراجع تقلید در عراق، مرجعیت شیعیان ایران و دیگر کشورها را عهده\u200cدار شد. اطلاعات بیشتر در این خصوص در کتاب "روزشمار تاریخ معاصر ایران" تالیف حسن فراهانی قابل دسترس است.', 'Q&A': [{'question': 'آیت\u200cالله حائری یزدی در چه سالی و در کجا متولد شد؟', 'answer': 'آیت\u200cالله حائری یزدی در سال ۱۲۷۶ قمری در روستای مهرجرد از توابع میبد یزد متولد شد.'}, {'question': 'آیت\u200cالله حائری یزدی چرا به حوزه علمیه نجف رفت؟', 'answer': 'ایشان برای ادامه تحصیل علوم دینی به 

  2%|▏         | 3/181 [00:07<07:34,  2.55s/it]

...

{'sentence': 'در سال ۱۹۱۷، قراردادی موسوم به "امتیاز خوشتاریا" بین دولت ایران و آکاکی خوشتاریا، یکی از اتباع روسیه، به امضا رسید. این قرارداد به خوشتاریا اجازه بهره\u200cبرداری از منابع ایران را می\u200cداد. وثوق\u200cالدوله، نخست\u200cوزیر وقت، این امتیازنامه را برای ۷۰ سال اعطا کرد. طبق این قرارداد، دولت ایران ۱۲ درصد از سود حاصل از بهره\u200cبرداری از منابع نفت در پنج ایالت شمالی را دریافت می\u200cکرد. این امتیازنامه، که در تهران منعقد شد، به دلیل اتهامات رشوه و افشای رازهای آن، چالش\u200cهایی را برای روابط ایران با شوروینیز به وجود آورد.', 'Q&A': [{'question': 'چه کسی امتیاز خوشتاریا را دریافت کرد؟', 'answer': 'آکاکی خوشتاریا، یکی از اتباع روسیه، این امتیاز را دریافت کرد.'}, {'question': 'امتیاز خوشتاریا در چه سالی به امضا رسید؟', 'answer': 'در سال ۱۹۱۷ میلادی این قرارداد امضا شد.'}, {'question': 'چرا این امتیازنامه برای روابط ایران با شوروی چالش\u200cبرانگیز بود؟', 'answer': 'به دلیل اتهامات رشوه و افشای رازهای امتیازنامه، این قرارداد روابط ایران و شوروی را دچار چالش کرد.'}, {'que

 99%|█████████▉| 180/181 [06:51<00:01,  1.96s/it]

{'sentence': 'در سال ۱۹۸۹، پیمان\u200cنامه حقوق کودک، یک پیمان بین\u200cالمللی که حقوق مدنی، سیاسی، اقتصادی، اجتماعی و فرهنگی کودکان را مشخص می\u200cکند، توسط مجمع عمومی سازمان ملل متحد تصویب شد. این پیمان\u200cنامه که از ۲ سپتامبر ۱۹۹۰ لازم\u200cالاجرا شده است، تاکنون توسط ۱۹۳ کشور (به جز ایالات متحده آمریکا) تصویب یا به آن ملحق شده است. امضای این پیمان\u200cنامه به معنای پذیرش تعهدات مندرج در آن توسط کشورهاست و دولت\u200cها موظف به اجرای آن هستند. این سند، اولین سند حقوق بشر است که در تاریخ به تصویب رسیده و به همین دلیل، از اهمیت تاریخی بالایی برخوردار است. رسیدگی به تخلفات و شکایت\u200cهای مربوط به این پیمان\u200cنامه، بر عهده کمیته حقوق کودک ملل متحد است.', 'Q&A': [{'question': 'چه زمانی پیمان\u200cنامه حقوق کودک توسط مجمع عمومی سازمان ملل متحد تصویب شد؟', 'answer': 'در سال ۱۹۸۹.'}, {'question': 'چرا پیمان\u200cنامه حقوق کودک از اهمیت تاریخی بالایی برخوردار است؟', 'answer': 'زیرا اولین سند حقوق بشر است که در تاریخ به تصویب رسیده است.'}, {'question': 'چه کشورهایی به پیمان\u200cنامه 

100%|██████████| 181/181 [06:53<00:00,  2.28s/it]

{'sentence': 'کنوانسیون مربوط به وضعیت پناهندگان، یک معاهده بین\u200cالمللی است که توسط سازمان ملل متحد در ۲۸ ژوئیه ۱۹۵۱ به تصویب رسید و در سال ۱۹۵۴ به اجرا درآمد. این کنوانسیون حقوق پناهندگان را مشخص می\u200cکند و مسئولیت\u200cهای کشورهای عضو را در قبال آنان تعیین می\u200cکند. در ابتدا، این کنوانسیون محدود به پناهندگان اروپایی پس از جنگ جهانی دوم بود، اما با گذشت زمان و در سال ۱۹۶۷، محدودیت\u200cهای جغرافیایی و زمانی آن برداشته شد و دامنه شمول آن گسترش یافت. هدف اصلی این کنوانسیون، تعریف پناهنده و تعیین حقوق و وظایف مربوط به حمایت از این افراد است. این سند مهم، از طریق وبسایت ویکی\u200cپدیا قابل دسترسی است.', 'Q&A': [{'question': 'کنوانسیون مربوط به وضعیت پناهندگان در چه تاریخی به تصویب رسید؟', 'answer': 'این کنوانسیون در ۲۸ ژوئیه ۱۹۵۱ توسط سازمان ملل متحد به تصویب رسید.'}, {'question': 'هدف اصلی کنوانسیون مربوط به وضعیت پناهندگان چه بود؟', 'answer': 'هدف اصلی این کنوانسیون تعریف پناهنده و تعیین حقوق و وظایف مربوط به حمایت از این افراد بود.'}, {'question': 'در ابتدا، کنوانسیون مربوط ب

In [ ]:
filename = "/content/drive/MyDrive/NLP_HW2/History/ReadableFormats/QA_records_final.json"

with open(filename, 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=2)